# Physics 300 
## Computational Physics I (Fall 2018)
## BPB-248, Mon/Wed 02:30 - 03:45 pm 

|Instructor| Prof. Qiang Zhu|
|--|-------------------------------|
|Email | qiang.zhu@unlv.edu|
|Website|http://www.physics.unlv.edu/~qzhu/|
|Office| BPB 232|
|Office hours | Mon/Wed 03:45 - 05:00 pm |

# 20 Global Optmization (III)

## 20.1 Genetic Algorithm

A genetic algorithm is a search heuristic that is inspired by Charles Darwin’s theory of natural evolution. This algorithm reflects the process of natural selection where the fittest individuals are selected for reproduction in order to produce offspring of the next generation.

- 1, Initialization
- 2, Selection
- 3, Genetic operators (crossover/mutation)
- 4, Termination


## 20.2 Application to LJ clusters


### Cambridge cluster database

Global optimization of LJ clusters has been one of the most interesting subject in computational physics/chemistry community. It has become a gold standard test bed when people wants to propose a new method.

Below is link to those reported geometry and energy values collected by Wales's group in Cambridge.

- [N = 1-150](http://doye.chem.ox.ac.uk/jon/structures/LJ/tables.150.html)
- [N = 310-561](http://doye.chem.ox.ac.uk/jon/structures/LJ/LJ310-561.html) 

In [1]:
from scipy.optimize import minimize
import numpy as np
from numba import jit
from time import time

@jit
def LJ(r):
    r6 = r**6
    r12 = r6*r6
    return 4*(1/r12 - 1/r6)

@jit
def total_energy(points): #points is a 3*N array of all the atoms involved
    total = 0
    num_atoms = int(len(points)/3)
    
    for i in range(num_atoms - 1):
        for j in range(i+1, num_atoms):
            pos1 = points[i*3 : (i+1)*3]
            pos2 = points[j*3 : (j+1)*3]
            
            r = np.linalg.norm(pos1-pos2)
            
            total += LJ(r)
    return total

@jit
def gradient(pos):
    """
    Calculate and return the Jacobian for minimizing
    the Lennard-Jones potential between N atoms.
    Input:
        positions: 3*N 1-D array of x,y,z positions of each atom
    Output:
        3*N 1-D array of the Jacobian elements for N atoms.
    """
    N = int(len(pos)/3)
    J = np.zeros([N, 3]) 
    
    for i in range(N-1):
        center = pos[3*i:(3*i)+3]
        for j in range(i+1, N):
            neighbor = pos[3*j:(3*j)+3] 
            r = np.linalg.norm(center - neighbor)
            J[i] += (24/r**7 - 48/r**13)*(center - neighbor)/r
    
    return J.flatten()
            
def init_pos(N, L=5):
    return L*np.random.random_sample((N*3,))

def init_population(population_size, N_atom):
    cluster = []
    for i in range(population_size):
        cluster.append(init_pos(N_atom))
    return np.array(cluster)

def local_optimize(cluster):
    fitness = []
    for i, cluster0 in enumerate(cluster):
        res = minimize(total_energy, cluster0, method='Powell', tol=1e-4, options={'maxiter':20}) #opt w/o gradient
        res = minimize(total_energy, res.x, method='CG', jac=gradient, tol=1e-4) #analytical gradient
        #res = minimize(total_energy, cluster0, method='CG', tol=1e-3) 
        cluster0 = res.x
        fitness.append(res.fun)
    return cluster, np.array(fitness)
        
def ranking(fitness):
    return np.argsort(fitness)

def mutation(cluster, rank, kT=0.5):
    #Needs to change
    id = int(len(rank)*np.random.random()/2)
    cluster0 = cluster[id]
    disp = np.random.random_sample((len(cluster0),))-0.5
                                   
    return cluster0 + kT*disp

def crossover(cluster, rank):
    id1 = int(len(rank)*np.random.random()/2)
    while True:
        id2 = int(len(rank)*np.random.random()/2)
        if id2 != id1:
            break
    frac = np.random.random()
    return cluster[id1]*frac + cluster[id2]*(1-frac)


def GA(generation_num=10, population_num=10, atom_num=10, ratio=0.7):
    for i in range(generation_num):
        t0 = time()
        if i == 0:
            cluster = init_population(population_num, atom_num)
            
        cluster, fitness = local_optimize(cluster)
        print(fitness)
        print('best fitness in generation ', i, ':  ', min(fitness), ' time:', time()-t0)
        
        rank = ranking(fitness)
        #selecton process
        new_cluster = []
        for j in range(population_num):
            if j < int(ratio*population_num):
                new_cluster.append(crossover(cluster, rank))
            else:
                new_cluster.append(mutation(cluster, rank))
        cluster = new_cluster

In [2]:
GA(generation_num=10, population_num=10, atom_num=13)

[-28.03445474 -21.5432442  -31.98071953 -25.8400955  -38.92840308
 -28.28758625 -30.8545103  -32.32523329 -33.44000424 -24.96460992]
best fitness in generation  0 :   -38.92840308199714  time: 9.605452060699463
[-35.68545056 -28.04876667 -36.22603736 -38.52120927 -36.74770683
 -35.46457623 -38.54372824 -35.10918799 -30.54869858 -34.44131591]
best fitness in generation  1 :   -38.543728237428866  time: 5.7271788120269775
[-38.3663331  -37.28950228 -35.82183588 -38.32168886 -37.45313246
 -37.24235484 -37.58509019 -37.22186267 -37.30010697 -25.82802117]
best fitness in generation  2 :   -38.36633309949232  time: 5.576938152313232
[-38.82222801 -38.75500461 -36.69034322 -37.6880879  -39.39480742
 -37.42866248 -37.4612133  -37.57912249 -38.05711089 -35.55053052]
best fitness in generation  3 :   -39.394807420216395  time: 6.313671827316284
[-38.26706692 -37.5866839  -37.33359313 -39.21446906 -39.50143472
 -37.72039081 -40.70891947 -38.48287034 -38.75587876 -39.53131135]
best fitness in gene

In [3]:
GA(generation_num=10, population_num=20, atom_num=13)

[-35.1212363  -37.51142547 -34.89847512 -38.11825645 -39.52454534
 -24.60495819 -38.36077056 -23.88349869 -34.45336567 -33.02207992
 -35.46541701 -36.76454891 -26.1661294  -31.99282699 -27.70450191
 -25.63728519 -35.30493895 -39.44480277 -38.02394609 -26.86253784]
best fitness in generation  0 :   -39.524545336318816  time: 10.941719055175781
[-29.77219158 -38.67924075 -36.4977469  -31.87887991 -38.75842563
 -38.82032588 -39.35844503 -35.11722159 -38.16491075 -38.87914239
 -30.38841697 -36.94390771 -33.40803486 -37.70916198 -21.98423016
 -29.76214635 -26.90597776 -31.52896488 -37.74504813 -40.08110408]
best fitness in generation  1 :   -40.08110407508179  time: 11.077372789382935
[-37.51976518 -37.51352132 -34.83580464 -37.76984572 -38.71598968
 -38.20085925 -36.88502225 -38.32815223 -37.56017979 -37.01805593
 -37.17586287 -32.61596231 -37.95999764 -36.00786558 -34.50510345
 -35.24711864 -36.14748825 -35.80779445 -30.0889079  -38.52811233]
best fitness in generation  2 :   -38.71598968

In [4]:
GA(generation_num=20, population_num=20, atom_num=13)

[-37.5733661  -34.59401873 -32.68057656 -35.32428474 -36.4291961
 -38.46501733 -33.6541149  -27.21996829 -28.11863055 -29.83286988
 -27.86200803 -37.32699554 -23.32377409 -36.2454605  -26.52682002
 -36.41479939 -33.988114   -29.5355677  -34.36103871 -30.9611333 ]
best fitness in generation  0 :   -38.46501733118478  time: 11.539275169372559
[-35.29813338 -37.26384989 -37.88742848 -23.67855637 -38.78308881
 -35.49185834 -34.81431395 -36.68546156 -37.43023352 -37.2184471
 -36.56121049 -30.47397672 -36.32645233 -23.88509329 -37.18992757
 -38.44415304 -22.48809033 -38.53876795 -22.67805969 -31.41552777]
best fitness in generation  1 :   -38.78308881437053  time: 11.737926244735718
[-36.06809965 -34.42474657 -37.61319855 -38.75685727 -38.54458802
 -34.56040099 -38.7257641  -41.46570321 -39.63982759 -39.38728937
 -37.24143074 -39.58799583 -39.64970299 -37.63022702 -38.67915809
 -36.25042139 -36.72571505 -39.21303847 -40.19566396 -38.68854366]
best fitness in generation  2 :   -41.46570321034

In [5]:
GA(generation_num=20, population_num=20, atom_num=13, ratio=0.1)

[-35.75487227 -26.55541219 -34.10252513 -35.32503859 -36.98180417
 -32.58483534 -34.61596126 -35.62974918 -28.04008102 -35.87657746
 -37.14016565 -29.43056337 -28.99267903 -24.10583741 -26.93366006
 -37.399122   -35.26391346 -36.59120487 -38.88817552 -35.39757958]
best fitness in generation  0 :   -38.888175515951175  time: 10.791239023208618
[-39.76870757 -38.88373895 -37.02175675 -21.50896562 -33.90840705
 -36.5207245  -30.5546796  -36.06879638 -30.06312736 -32.21654878
 -33.21979135 -28.94879893 -33.15834735 -25.18600902 -27.04140373
 -36.29249558 -27.33335841 -28.95810893 -35.46453921 -38.6139645 ]
best fitness in generation  1 :   -39.768707568390596  time: 11.049476146697998
[-38.59651087 -39.63952741 -33.33728143 -38.05157136 -39.42330187
 -35.18606082 -38.85679922 -35.82563542 -31.61476866 -36.59027962
 -21.35836077 -36.9643573  -34.4669797  -32.24515072 -29.39203163
 -36.33732222 -35.0400286  -32.14852729 -18.60416302 -31.30317478]
best fitness in generation  2 :   -39.6395274

## 20.3 Homework

Check the following documentation and use the differential evolution method to perform optimization on LJ clusters
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html#scipy.optimize.differential_evolution